# Classificação - Concrete Crack
Banco de dados utilizado: https://data.mendeley.com/datasets/5y9wdsg2zt/2

## Processamento das Imagens

**Bibliotecas Necessárias**
- os - Acessar arquivos do computador
- numpy - Funções Algébricas e mexer com Matrizes
- matploitlib - Gráficos
- time - Tempo de execução
- PIL - Modificar imagens
- sklearn.model_selection - Dividir dados entre teste e treino
- numba - Agilizar processamento de dados

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import time
from PIL import Image
from sklearn.model_selection import train_test_split
from numba import njit, prange

**Processamento das Imagens**
1. Redimensiona para resolução de 32x32
2. Transforma a imagem cinza
3. Transforma em array do tipo float
4. Transforma o array em uma linha (coluna abaixo de coluna)
5. Normaliza os valores
6. Adiciona o array embaixo da matriz dados


In [ ]:
def obter_dados(input_dir):
 
    # Obtém lista de arquivos do diretório
    lista_de_arquivos = os.listdir(input_dir)
    
    # Obtém um vetor com lixo de memória para imagens serem adicionadas lado a lado
    dados = np.empty(shape=[1, 32*32], dtype=int)
 
    for arquivo in lista_de_arquivos:
 
        # Abre imagem do diretório
        imagem = Image.open(os.path.join(input_dir, arquivo))
 
        # Redimensiona imagem
        imagem_redimensionada = imagem.resize((32, 32))
 
        # Torna imagem redimensionada cinza
        # .convert('L') utiliza somente a luminosidade, converte para tons de cinza
        imagem_cinza = imagem_redimensionada.convert('L')
 
        # Transforma imagem em array com valores em float
        imagem_array = np.asarray(imagem_cinza, dtype='float')
        
        # Normaliza os valores para ficar entre valores entre 0 e 1
        imagem_array /= 255.0
 
        # Ordena coluna embaixo de coluna e transforma em uma linha
        uma_linha = imagem_array.reshape((1, 1024), order='F')
 
        # Adiciona nova linha embaixo da matriz dados
        dados = np.vstack((dados, uma_linha))
        
    # Retorna dados e deleta primeira linha que possui valores lixo de memória
    return np.delete(dados,(0), axis=0)
 
 
diretorio_negative = '/Content/Concrete_crack/Negative'
diretorio_positive = '/Content/Concrete_crack/Positive'
 
dados_positive = obter_dados(diretorio_positive)
dados_negative = obter_dados(diretorio_negative)
 
# Junta dados com rachadura e sem rachadura
dados = np.vstack((dados_positive, dados_negative))
 
# Salva matriz de dados amostras x características
np.save(os.path.join('Content/Concrete_crack', 'dados.npy'), dados)

Com a matriz dados salva em um arquivo numpy, podemos caregá-la rapidamente abrindo tal arquivo.

In [ ]:
# Carregar matriz dados a partir do arquivo dados.npy
dados = np.load('/content/Concrete_crack/dados.npy')

# Imprime matrizes obtidas
print(f'Matriz dados:\n\n{dados}\n\nDimensões: {dados.shape}')
print('Data Type: %s' % dados.dtype)
print('Min: %.3f, Max: %.3f' % (dados.min(), dados.max()))

Calculamos a matriz y, que armazena os resultados, com 1 para imagens com rachaduras e 0 para imagens sem rachaduras.

Dividimos as amostras em treino e teste (proporção de 60%/40%). 

A partir de agora, usaremos apenas a matriz X_train (treino) para mostrarmos diversas propriedades das matrizes ortogonais e das projeções ortogonais.

In [ ]:
# Matriz y (resultados)
# 1 = Positivo para Rachadura
y = np.ones(20000).reshape(-1, 1)
# 0 = Negativo para Rachadura
y = np.vstack((y, np.zeros(20000).reshape(-1,1)))

# Dividir dados em teste e treino
X_train, X_test, y_train, y_test = train_test_split(dados, y, test_size=0.40, random_state=0)

# Imprime dimensões das matrizes obtidas
print(f'Dimensões:\n\nX treino: {X_train.shape}\nX teste: {X_test.shape}')
print(f'\ny treino: {y_train.shape}\ny teste: {y_test.shape}')

## Tarefas


### 1. Centralizar a matriz de dados original $\tilde{X}$ obtendo a matriz X.


Resolução:

Para centralizar a matriz, calculamos o valor médio de cada variável, localizadas nas colunas, com a função numpy.mean, tendo como argumento axis=0, o qual indica que será calculada a média dos valores das colunas e em seguida subtraímos do conjunto de dados.

In [ ]:
# Centraliza-se os dados em relação à média da respectiva coluna

X_train_mean = np.mean(X_train, axis=0)
X_train = X_train - X_train_mean

X_test_mean = np.mean(X_test, axis=0)
X_test = X_test - X_test_mean

### 2. Calcular a matriz de covariância dos dados de duas maneiras:
- $np.cov(\tilde{X}$),
- $\frac{1}{m-1}X^TX$

Comparar os resultados. Atenção: verifique se o parâmetro rowvar na
função np.cov( ) deve ser True ou False.


Resolução:

Para calcularmos a matriz de covariância, comparamos duas maneiras, na primeira utilizamos a função numpy.cov nos dados não centralizados, com o paramêtro rowvar=False, já que as variáveis estão localizadas nas colunas, e na outra forma utilizamos a fórmula $\frac{1}{1-m}X^TX$ nos dados centralizados. 

A Matriz resultante da subtração das duas matrizes de covariância, caso essas sejam iguais, deve dar próxima da matriz nula. Para verificarmos, calculamos o valor absoluto máximo de tal matriz, que deu 2.9976021664879227e-15, ou seja, um número muito pŕoximo de zero, e uma diferença desprezível resultante da imprecisão do tipo float.

Visto isso, os dois métodos são viáveis.

In [ ]:
# Matriz covariância utilizando np.cov
matriz_covariancia_1 = np.cov(X_train + X_train_mean, rowvar=False)

# m = numero de linhas
m = np.size(X_train, 0)

fator = 1 / (m - 1)

# Matriz covariância utilizando fórmula algébrica
matriz_covariancia_2 = fator * np.transpose(X_train) @ X_train

print(f'Matriz Covariância utilizando np.cov:\n\n{matriz_covariancia_1}\n')
print(f'Matriz Covariância utilizando fórmula algébrica:\n\n{matriz_covariancia_2}\n')

# Valor absoluto máximo da subtração das duas matrizes
# Para comparação das matrizes
maximo_absoluto = np.max(np.abs(matriz_covariancia_1 - matriz_covariancia_2))

print(f'\nNúmero máximo - diferença: {maximo_absoluto}')

### 3. Calcular a decomposição espectral da matriz de covariância dos dados, isto é,  $cov(X) = QΛQ^T$, usando a função do NumPy: np.linalg.eigh(cov(X)).



Resolução:

Para calcular a decomposição espectral, ou seja, obter os autovalores e autovetores, usamos a função np.linalg.eigh, com a matriz de covariância como parâmetro.

Após obtermos os autovalores e autovetores, ordenamos eles de acordo com a  ordem decrescente dos autovalores, que fora obtido calculando-se a ordem inversa do retorno da função np.argsort, a qual ordena a matriz em ordem crescente.

In [ ]:
# Obtém os autovalores e autovetores
w, Q = np.linalg.eigh(np.cov(X_train, rowvar=False))

# Obtém os índices para ordenação decrescente dos autovalores
indices = np.argsort(w)[::-1]

# Ordena autovalores e autovetores em ordem decrescente
w = w[indices]
Q = Q[indices]

print(f'Autovalores:\n\n{w}\n')
print(f'Autovetores:\n\n{Q}')
print(f'Tempo de execução: {time.time()-inicio}')

## Questões

2. Vericar, com o respectivo banco de dados, que $Q^TQ ≈ I.$


Resolução:

Para verificarmos a que Q realmente é ortogonal, ou seja, $Q^TQ ≈ I$, calculamos a matriz diferença entre $Q^TQ$ e $I$, que deve resultar em uma matriz próxima da nula, caso essas matrizes sejam muito pŕoximas. 

A fim de verificarmos isso, calculamos o número absoluto máximo presente na matriz diferença, 1.0436096431476471e-14, o qual, por ser muito próximo de zero, com uma diferença desprezível resultante da imprecisão do tipo float, indica que as matrizes são muito parecidas.

Visto isso, a matriz Q é ortogonal.

In [ ]:
# Calcula Q transposto Q
teste_ortogonal = np.transpose(Q) @ Q 

# Identidade de mesmas dimensões
identidade = np.identity(1024, dtype='float')

# Matriz teste se tais matrizes são iguais/parecidas
maximo_absoluto = np.max(np.abs(teste_ortogonal - identidade))

print(f'Número máximo - diferença: {maximo_absoluto}')

### 5. Calcular, com o respectivo banco de dados, as matrizes $\hat{Z}$ e $\hat{X}$; vericar que a matriz de projeção $$\hat{Q}\hat{Q}^T$$ **não** é a matriz identidade.

Resolução:

Para calculamos $\hat{Q}$, deletamos metade das colunas de Q, obtendo uma matriz de dimensões 1024x512. 

Já em $\hat{Z}$, utilizamos a fórmula $\hat{Z} =  X\hat{Q}$, obtendo uma matriz de dimensões 24000x512

Em $\hat{X}$, utilizamos a fórmula $\hat{X} = X\hat{Q}\hat{Q}^T$, obtendo uma matriz de dimensões 24000x1024

Com o objetivo de verificarmos se $\hat{Q}$ é ortogonal, subtraímos a matriz $\hat{Q}^T\hat{Q}$ da matriz identidade $I$, e obtemos como número absoluto maxímo da matriz diferença 0.5660154821287144, o que indica que $\hat{Q}^T\hat{Q}$ não é a matriz identidade

In [ ]:
# Deleta metade das colunas de Q (colunas de índices pares)
Q_chapeu = np.delete(Q,np.s_[::2], 1)

# Calcula Z chapeu
Z_chapeu = X_train @ Q_chapeu

# Calcula X chapeu
X_chapeu = X_train @ Q_chapeu @ np.transpose(Q_chapeu)

# Calcula Qchapeu * Qchapeu transpoto
teste_ortogonal = Q_chapeu @ np.transpose(Q_chapeu) 

# Identidade de mesmas dimensões
identidade = np.identity(1024, dtype='float')

# Matriz teste se tais matrizes são iguais/parecidas
teste_igualdade = teste_ortogonal - identidade

# Resultados
print(f'Matriz Q chapéu: \n\n{Q_chapeu}\nDimensões: {Q_chapeu.shape}\n')
print(f'Matriz Z chapéu: \n\n{Z_chapeu}\nDimensões: {Z_chapeu.shape}\n')
print(f'Matriz X chapéu: \n\n{X_chapeu}\nDimensões: {X_chapeu.shape}\n')
print(f'\nVerificação Q chapéu * Q chapéu tranposto\n\nNúmero máximo - diferença: {np.max(np.abs(teste_igualdade))}')

# Trabalho Final

## Tarefas

### 01. Calcular, para o respectivo banco de dados, a SVD da matriz de dados centralizados X. Gerar um gráfico de número de valores singulares versus variabilidade acumulada.


Para realizar a decomposição em valores singulares(SVD), utilizamos a função np.linalg.svd. Após receber as matrizes U, S e V transposto, ordenamos S em ordem decrescente e, a partir disso, ordenamos a matriz V. Os resultados foram uma matriz U de 24000 por 1024, um vetor S de 1024, o qual representa a diagonal principal da matriz que contém os autovalores na decomposição svd, e a matriz V transposta de 1024 por 1024.

Para calcular a variabilidade acumulada, realizamos a divisão entre a soma acumulada de autovalores e a soma de dos mesmos, utilizando a  função np.cumsum de S ao quadrado e a função np.sum de S ao quadrado, e geramos um gráfico utilizando a biblioteca matplotlib, de valores singulares X variabilidade acumulada. 

In [ ]:
# Calcula a SVD da matriz
U, S, Vt = np.linalg.svd(X_train, full_matrices=False)

# Obtém os índices para ordenação decrescente dos valores singulares de S
indices = np.argsort(S)[::-1]

# Ordena S e V em ordem decrescente
S = S[indices]
Vt = Vt[indices,:]

print('Dimensões: \nU: {} \nS: {} \nVt: {}' \
      .format(U.shape, S.shape, Vt.shape) )

In [ ]:
# Array com: Soma acumulada / Soma de valores singulares
variabilidade_acumulada = np.cumsum(S**2) / np.sum(S**2)

# Número de autovalores
x_plot = range(len(S))

# Criar Gráfico
plt.plot(x_plot, 100 * variabilidade_acumulada)
plt.xlabel('Numero de Valores Singulares')
plt.ylabel('Variabilidade Acumulada')
plt.grid(True)
plt.show()

### 02. Selecionar valores apropriados de variabilidade acumulada para reduzir a dimensionalidade do problema de classificação, resolvendo-o para o respectivo banco de dados.

Para realizar a redução de dimensionalidade, calculamos o número de autovalores necessários para uma porcentagem de 95% em variabilidade acumulada, utilizando compreensão de listas e a função enumerate. Como resultado, obtivemos que 185 valores singulares eram necessários para tal objetivo.

Com isso, calculamos a nova matriz V, com apenas 185 colunas, e calculamos a projeção da matriz X_train(24000x1024) e X_test(16000x1024) na nova V. Assim, obtivemos uma matriz Y_train de 24000 por 185 e uma Y_test de 16000 por 185

In [ ]:
porcentagem = 0.95

# Armazena o primeiro elemento da lista do número de valores singulares
# para variabilidade acumulada > porcentagem
r = [n for n, i in enumerate(variabilidade_acumulada) if i > porcentagem][0]

print( 'Variabilidade acumulada de {:.2f}% com {} Valores Singulares\n' \
      .format(100 * variabilidade_acumulada[r], r) )

# Redução de dimensionalidade
V_chapeu = Vt[:r,:].T 
 
Y_test = X_test @ V_chapeu

Y_train = X_train @ V_chapeu

print( 'Dimensões Antes:\nX_train: ', X_train.shape, \
       '\nX_test: ', X_test.shape, \
       '\nV: ', Vt.shape)

print( '\nDimensões Depois:\nY_train: ', Y_train.shape, \
       '\nY_test: ', Y_test.shape, \
       '\nV_chapeu: ', V_chapeu.shape)

Para classificar as imagens, utilizamos o método kNN (k vizinhos mais próximos), considerando k=1, o qual consiste em achar o vetor mais próximo da imagem de teste que queremos classificar. Para isso, utilizamos a função norm do numpy.linalg com o argumento axis=1, que retorna um array com as diferenças entre o vetor teste e todos os vetores do X_train. 

Após isso, armazenamos a posição da menor distância com a função np.argmin, e verificamos o resultado comparando o y_train do vetor mais parecido com o y_test do vetor que estamos querendo classificar. Por fim, temos o número de classificações corretas e podemos verificar a acurácia. 

Utilizando uma variabilidade acumulada de 95% e a redução de dimensionalidade com 185 autovalores, foi obtida uma acurácia de 93.50%.

In [ ]:
# Classificação de Imagens

corretos = 0
error_ids = []
for i, vetor_test in enumerate(Y_test):
    # Distância entre cada vetor da matriz de treino e o vetor da matriz de teste
    distances = np.linalg.norm(Y_train - vetor_test, axis=1)
    # Vetor mais parecido
    mais_parecido = np.argmin(distances)
    # Verifica Resultado
    if y_train[mais_parecido] == y_test[i]:
        corretos += 1
    else:
        error_ids.append(i)
        
print('Acurácia: {:.2f}%'.format(100 * corretos/16000))

Exemplos do resultado da classificação:

In [ ]:
# Funções - Exemplo de Classificação

# Plota o gráfico com a imagem real e a mais parecida
def comparar_imagens(real, mais_parecido):
    fig = plt.figure(figsize=(8,5))
    fig.suptitle('Comparação de Resultados', fontsize=14)

    fig.add_subplot(1,2,1)
    plt.title('Imagem original')
    plt.imshow(real, cmap='gray')
    plt.axis('off')

    fig.add_subplot(1,2,2)
    plt.title('Imagem mais parecida')
    plt.imshow(mais_parecido, cmap='gray')
    plt.axis('off')

    plt.show()

    
# Imprime se a imagem contém rachadura ou não
# y = 1 -> Contém rachadura
# y = 0 -> Não contém rachadura
def imprimir_resultados(id, pos):

    resultados = []
    
    if y_test[id] == 1:
        resultados.append('Contém Rachadura')
    else:
        resultados.append('Não Contém Rachadura')
        
    if y_train[pos] == 1:
        resultados.append('Contém Rachadura')
    else:
        resultados.append('Não Contém Rachadura')

    print('Resultados:\nImagem original: ', resultados[0], \
          '\nImagem mais parecida: ', resultados[1])

    
# Classifica a imagem e mostra os resultados
def classificar(id):
    
    vetor_test = Y_test[id]
    
    # Distância entre cada vetor da matriz de treino e o vetor da matriz de teste
    distances = np.linalg.norm(Y_train - vetor_test, axis=1)
    
    # Vetor mais parecido
    pos = np.argmin(distances)
    
    # Restaura a média
    X_train_image = X_train + X_train_mean
    X_test_image = X_test + X_test_mean

    # Vetores do resultado
    mais_parecido = X_train_image[pos]
    real = X_test_show[id]
    
    # Torna os vetores 32x32
    mais_parecido = mais_parecido.reshape((32, 32))
    real = real.reshape((32,32))
    
    # Gŕafico com resultados
    comparar_imagens(real,mais_parecido)
    # Resultados
    imprimir_resultados(id, pos)

In [ ]:
# Exemplo - resultado correto
classificar(2)

In [ ]:
print('Exemplo de erros: ', error_ids[:5])

In [ ]:
# Exemplo - resultado errado
classificar(108)

### 03. Gerar um gráco de número de valores singulares versus acurácia. Isso deve ser feito de maneira apropriada, não devendo o gráco ser gerado em tempo superior a 24 horas.

Para gerar o gráfico de valores singulares X acurácia, definimos duas funções, a componentes_principais, que recebe o número de autovalores para a diminuição de dimensionalidade e retorna as novas matrizes Y_train e Y_test reduzidas, e a find_match, que utiliza a biblioteca numba para um processamento mais rápido e realiza a classificação, recebendo como argumento as matrizes reduzidas, as matrizes ytrain e ytest, que armazenam as classes de cada imagem, o número de vetores teste e o número de vetores treino, e retorna a acurácia da classificação.

De posse dessas funções, realizamos a classificação no range de 1 a 1024 valores singulares, pulando de 5 em 5, e guardamos a acurácia em uma lista, processo realizado em 23 horas, 32 minutos e 36 segundos. Após o processamento, foi possível criar o gráfico de valores singulares X acurácia.

In [ ]:
def componentes_principais(r):
    V_chapeu = Vt[:r,:].T 
    Y_test = X_test @ V_chapeu
    Y_train = X_train @ V_chapeu
    
    return Y_train, Y_test

@njit(fastmath=True, cache=True)
def find_match(Ytrain, Ytest, ytrain, ytest, ntrains, ntests):
    count = 0
    for i in range(ntests):
        min_dist = 10000
        for j in range(ntrains):
            dist = np.linalg.norm(Ytest[i,:] - Ytrain[j,:])
            if dist <= min_dist:
                index = j
                min_dist = dist
        if ytrain[index] == ytest[i]:
            count += 1
    return count / ntests

acuracia = []

inicio = time.time()

for i in range(1, 1024, 5):
    Y_train, Y_test = componentes_principais(i)
    count = find_match(Y_train, Y_test, y_train, y_test, Y_train.shape[0], Y_test.shape[0])
    acuracia.append(count)

tempo_total = time.time() - inicio

print('Tempo de execução: ', time.strftime('%H:%M:%S', time.gmtime(tempo_total)))

In [ ]:
# Gráfico
y_plot = acuracia
x_plot = list(range(1,1024,5))

plt.plot(x_plot, y_plot)
plt.xlabel('Número de valores singulares')
plt.ylabel('Acurácia(%)')
plt.grid(True)
plt.show()

## Questões

### 02. Vericar numericamente a proposição anterior, calculando, para o respectivo banco de dados, os autovalores de $X^TX$ e $XX^T$

Com o objetivo de obter um melhor processamento, utilizamos apenas as primeiras 5000 colunas da matriz de treino, e calculamos e ordenamos em ordem decrescente as matriz de autovalores e autovetores das matrizes $X^TX$ e $XX^T$

Para obter a matriz sem os autovalores nulos de $XX^T$, armazenamos apenas as 1024 primeiras colunas (as unícas com valores não nulos, após ordenamento decrescente). 

Afim de verificar a proposição da questão 1, subtraímos a matriz de autovalores de $X^TX$ com a matriz de autovalores não nulos de $XX^T$, e obtivemos uma diferença de 2.9103830456733704e-11, desprezível, resultante da impressão dos cálculos. Portanto, os autovalores não nulos são iguais.

In [ ]:
# Calcula os autovalores e vetores de xxt e xtx:
X=X_train[:5000:]

w1, q1 = np.linalg.eigh(X.T @ X)
w2, q2 = np.linalg.eigh(X @ X.T)

# Ordena os autovalores em ordem decrescente:
indices1 = np.argsort(w1)[::-1]
autovalores_decrescentes1 = w1[indices1]

indices2 = np.argsort(w2)[::-1]
autovalores_decrescentes2 = w2[indices2]

# Exclui os valores nulos da matriz xxt
autovalores_decrescentes2 = autovalores_decrescentes2[:1024:]

#armazena os dados e imprime o resultado
print(f'Autovalores XtX:\n{autovalores_decrescentes1}\n')
print(f'Aautovalores XXt:\n{autovalores_decrescentes2}\n')
print(f'Diferença: {np.max(np.abs( autovalores_decrescentes1 - autovalores_decrescentes2))}')

### 04. Vericar a proposição anterior, comparando, para o respectivo banco de dados, a matriz de autovetores de $X^TX$ e a matriz de vetores singulares direitos, isto é, $V$ em $X = USV^T$. Atenção para com os sentidos dos vetores.

Para provarmos que $X^TX$ tem os mesmo autovetores de V provido pela SVD, calculamos os autovetores(Q) com a função np.linalg.eigh, e ordenamos os valores absolutos de ambas as matrizes V e Q. Após isso, calculamos a diferença com a função np.max, que retornou 1.2541135664678738e-10, um número muito pequeno, provido da imprecisão dos cálculos.

In [ ]:
# Obtém os autovalores e autovetores
w, Q = np.linalg.eigh(X_train.T @ X_train)

# Utiliza o V provido da SVD
# Ordena os valores absolutos de ambas as matrizes
V = Vt.T
V  = np.sort(np.abs(V), axis=1)
Q = np.sort(np.abs(Q), axis=1)

# Resultado - Diferença
print(f'Diferença = {np.max(np.abs(V - Q))}')

### 05. Comparar, para o respectivo banco de dados, a matriz $Q$ de autovetores de $XX^T$ e a matriz de vetores singulares esquerdos, isto é, $U$ em $X = USV^T$. Justique por que as duas matrizes são diferentes, mas as submatrizes $U[:, 0 : t]$ e $Q[:, 0 : t]$ são iguais (t é o índice do primeiro autovalor nulo).

Reduzindo a matriz de treino para 3000x1024, para fins de melhor processamento, foi realizada a svd da matriz completa, para obter U de 3000x3000, e a decomposição espectral de $XX^T$ para obter o Q de 3000x3000. Após isso, foi realizado o ordenamento dos valores absolutos de ambas as matrizes, e calculada a diferença entre elas, resultando em 0.8868200676499786 e a diferença entre as submatrizes utilizando t = 1024, 0.006740097275460629.

In [ ]:
X = X_train[:3000:]

# SVD completa
u, s, vt = np.linalg.svd(X)

# Decomposição espectral
w, q = np.linalg.eigh(X @ X.T)

# Ordenamento
u = np.sort(np.abs(u), axis = 1)
q = np.sort(np.abs(q), axis = 1)

print(f'Diferença matrizes: {np.max(u-q)}')

u = u[:,0:1024]
q = q[:,0:1024]

print(f'Diferença submatrizes: {np.max(u-q)}')